In [1]:
! pip install openai

In [2]:
from openai import OpenAI
import json
import re

In [3]:
# Connect to your local Ollama server
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [4]:
# --- Local tools ---
def calculator(expression: str) -> str:
    """Safe math calculator."""
    try:
        allowed_chars = "0123456789+-*/(). "
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def local_search(query: str) -> str:
    """Pretend local knowledge base."""
    data = {
        "capital of blabla": "Blablaville",
        "atown to btown": "3,840 km",
    }
    return next((v for k, v in data.items() if k.lower() in query.lower()), "No results found in local database.")

TOOLS = {
    "calculator": calculator,
    "local_search": local_search,
}

In [5]:
# --- Agent system prompt ---
SYSTEM_PROMPT = """
You are a local reasoning agent that can use tools.
You have no internet access.
You call the following tools to answer questions step by step.

Tools:
- calculator(expression)
- local_search(query)

Finish after one step, either with a tool call or the final answer.
Only respond with a JSON object with the following format:
If you used a tool, respond in JSON:
{"action": "tool_name", "input": "..."}
or, when done:
{"action": "final_answer", "input": "..."}
"""

In [6]:
query = "What is the distance from Earth to the Moon divided by 1000?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": query},
]

In [7]:
def run_local_agent(query: str, max_steps: int = 5):
    print(f"🧠 User: {query}")
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]

    for step in range(max_steps):
        # Short-term memory:
        # If there's a tool result from last step, add it.
        if step > 0 and 'result' in locals():
            messages.append({
                "role": "user",
                "content": "The tool returned: " + result + 
                        "\nWhat should you do next? If you have enough information, return the final answer."
            })

        # Ask local model
        response = client.chat.completions.create(
            model="llama3",  # or any local model you pulled
            messages=messages,
        )
        content = response.choices[0].message.content.strip()
        print(f"Step {step+1} → {content}\n")

        # Parse tool or final answer
        tool_match = re.search(r'\{.*\}', content)
        if tool_match:
            json_str = tool_match.group(0)
            try:
                action = json.loads(json_str)
                name = action["action"]
                arg = action["input"]
            except Exception:
                print(f"⚠️ Invalid JSON from model: {json_str}")
                break
        else:
            print(f"⚠️ No JSON action found in model response.")
            break

        if name == "final_answer":
            print(f"✅ Final Answer: {arg}")
            break

        elif name in TOOLS:
            result = TOOLS[name](arg)
            print(f"🧰 {name}('{arg}') → {result}\n")
            messages.append({"role": "assistant", "content": content})
        else:
            print(f"⚠️ Unknown tool '{name}'")
            break
    else:
        print("⏹️ Reached max reasoning steps.")

In [8]:
run_local_agent("What is (15 + 5) * 3?")

🧠 User: What is (15 + 5) * 3?
Step 1 → {"action": "calculator", "input": "(15 + 5) * 3"}

🧰 calculator('(15 + 5) * 3') → 60

Step 2 → {"action": "final_answer", "input": "60"}

✅ Final Answer: 60


In [9]:
run_local_agent("What is the distance from Atown to Btown divided by 1000?")

🧠 User: What is the distance from Atown to Btown divided by 1000?
Step 1 → {"action": "calculator", "input": "distance from Atown to Btown / 1000"}

🧰 calculator('distance from Atown to Btown / 1000') → Error: Invalid characters

Step 2 → Since the calculator tool returned an error, I don't have enough information to provide a correct calculation.

{"action": "local_search", "input": "distance from Atown to Btown"}

🧰 local_search('distance from Atown to Btown') → 3,840 km

Step 3 → Now I can use the result to calculate the desired value.

{"action": "calculator", "input": "3840 / 1000"}

🧰 calculator('3840 / 1000') → 3.84

Step 4 → I have the final result!

{"action": "final_answer", "input": "3.84"}

✅ Final Answer: 3.84
